## Chassisml Example Notebooks
Welcome to the examples section for [Chassis.ml](https://chassis.ml), which contains notebooks that leverage Chassisml to auto-containerize models built using the most common machine learning frameworks. 

**NOTE:** Chassisml provides two key functionalities: 
1. Create a Docker container from your model code and push that container image to a Docker registry. This is the default behavior.
2. Should you pass valid Modzy credentials as optional parameters, Chassisml will take the container and upload it directly to the Modzy environment you specify. You will notice most of these notebooks deploy the model to one of the Modzy internal development environments.   

Can't find the framework you are looking for or need help? Fork this repository and open a PR, we're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/tdfXFY2y), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly. ! 

In [1]:
import chassisml
import pickle
import cv2
import torch
import getpass
import numpy as np
import torchvision.models as models
from torchvision import transforms

## Enter credentials
Dockerhub creds and Modzy API Key

In [12]:
print(chassisml.__version__)

1.3.0


In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

docker hub username········
docker hub password········
modzy api key········


## Prepare model, data transform object, and labels
Initialize anything here that should persist across inference runs

In [2]:
model = models.resnet50(pretrained=True)
model.eval()

labels = pickle.load(open('./data/imagenet_labels.pkl','rb'))

transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])        

# use GPU:
device = 'cpu'
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Write batch process function

* Must take list[bytes] input
* Preprocess all inputs, run inference in batch, postprocess batch model output, return list of formatted results

In [6]:
def batch_process(inputs):
    
    # preprocess list of inputs
    print(type(inputs))
    images = []
    for input_bytes in inputs:
        decoded = cv2.imdecode(np.frombuffer(input_bytes, np.uint8), -1)
        resized = cv2.resize(decoded, (224, 224)).reshape((1,224,224,3))
        images.append(resized)
    images_arr = np.concatenate(images)
    batch_t = torch.stack(tuple(transform(i) for i in images_arr), dim=0).to(device)

    # run batch inference and softmax
    output = model(batch_t)
    probs = torch.nn.functional.softmax(output, dim=1)
    softmax_preds = probs.detach().cpu().numpy()
    
    # postprocess
    all_formatted_results = []
    for preds in softmax_preds: 
        indices = np.argsort(preds)[::-1]
        classes = [labels[idx] for idx in indices[:5]]
        scores = [float(preds[idx]) for idx in indices[:5]]
        preds = [{"class": "{}".format(label), "score": round(float(score),3)} for label, score in zip(classes, scores)]
        preds.sort(key = lambda x: x["score"],reverse=True)
        results = {"classPredictions": preds}
        all_formatted_results.append(results)
    
    # output list of formatted results
    return all_formatted_results

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [4]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires at least one of single input `process_fn` or batch input `batch_process_fn` defined above
    * If you provide `batch_process_fn`, you must also provide a `batch_size`

In [9]:
# create Chassis model
chassis_model = chassis_client.create_model(batch_process_fn=batch_process,batch_size=4)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/airplane.jpg'
results = chassis_model.test(sample_filepath)
print(results)

<class 'list'>
b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}'


In [10]:
# test batch locally
results = chassis_model.test_batch(sample_filepath)
print(results)

<class 'list'>
[b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}', b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}', b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack aircraft carrier","score":0.048}]}', b'{"classPredictions":[{"class":"airliner","score":0.606},{"class":"crane","score":0.11},{"class":"wing","score":0.103},{"class":"chain saw, chainsaw","score":0.07},{"class":"aircraft carrier, carrier, flattop, attack a

## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, required Modzy info, AND specify gpu=True

In [25]:
MODZY_URL = "https://integration.modzy.engineering/api"

response = chassis_model.publish(model_name="Torch Imagenet GPU",model_version="0.0.1",
                     registry_user=dockerhub_user,registry_pass=dockerhub_pass,
                     modzy_sample_input_path=sample_filepath,
                     modzy_api_key=modzy_api_key,modzy_url=MODZY_URL, gpu=True)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Starting build job... Ok!


## Run sample job
Submit inference job to our newly-deploy model running on Modzy

In [ ]:
from modzy import ApiClient

client = ApiClient(base_url='https://integration.modzy.engineering/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

# submit 16 inputs
sources = {"input_{}".format(i): {input_name: sample_filepath} for i in range(16)}
inference_job = client.jobs.submit_file(model_id, model_version, sources)
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
print(inference_job_result["results"])